## 라이브러리 의존성

In [ ]:
!pip install elasticsearch pandas

from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import BulkIndexError
import pandas as pd

## 일래스틱서치 접속

In [ ]:
# 일래스틱서치 클라이언트 초기화. HOST:PORT/USERNMAME/PASSWORD는 사용하는 일래스틱서치 서버의 정보로 변경 필요
es = Elasticsearch(
    ['HOST:PORT'],
    basic_auth=('USERNMAME', 'PASSWORD'),
    verify_certs=False
)

## 데이터 세트 파싱

In [ ]:
# 인덱스 정의
index_name = 'recipes'

# 매핑 정의
mapping = {
    "mappings": {
        "properties": {
            "group": {"type": "text"},
            "name": {"type": "text"},
            "rating": {"type": "text"},
            "n_rater": {"type": "text"},
            "n_reviewer": {"type": "text"},
            "summary": {
                "type": "text",
                "analyzer": "english"
            },
            "process": {"type": "text"},
            "ingredient": {
                "type": "text"
            },
            "ml.tokens": {
                "type": "rank_features"
            }
        }
    }
}

# 인덱스 생성
es.indices.create(index=index_name, body=mapping)

# 판다스 라이브러리로 CSV 파일 불러오기(allrecipes.csv는 kaggle에서 다운로드 후 colab 폴더에 업로드 필요)
with open('allrecipes.csv', 'r', encoding='utf-8', errors='ignore') as file:
    df = pd.read_csv(file)

# allrecipes.csv의 rating열의 결측값 변경
df['rating'] = df['rating'].fillna(0)

# 색인을 위해 데이터프레임을 파이썬 딕셔너리 리스트로 변환
recipes = df.to_dict('records')
print(f"Number of documents: {len(recipes)}")

## 대량 색인

In [ ]:
# 대량 색인을 위한 JSON 문서 배열 생성(키바나에서 elser 모델 설치 및 파이프라인 생성 필요)
bulk_index_body = []
for index, recipe in enumerate(recipes):
    document = {
        "_index": "recipes",
        "pipeline": "elser-v2-recipes",
        "_source": recipe
    }
    bulk_index_body.append(document)

# 대량 색인 및 BulkIndexError 처리
try:
    response = helpers.bulk(es, bulk_index_body, chunk_size=500, request_timeout=60 * 3)
    print("\nRESPONSE:", response)
except BulkIndexError as e:
    for error in e.errors:
        print(f"Document ID: {error['index']['_id']}")
        print(f"Error Type: {error['index']['error']['type']}")
        print(f"Error Reason: {error['index']['error']['reason']}")